# Install Packages

In [1]:
%matplotlib inline

In [2]:
from __future__ import print_function
from collections import defaultdict
from datetime import datetime
import pandas
import pickle
from pprint import PrettyPrinter
import pymongo

In [3]:
printer = PrettyPrinter()

# Data Check

In [4]:
conn=pymongo.MongoClient('mongodb', 27017)

conn.database_names()

['admin', 'iati', 'local']

In [5]:
db = conn.iati

activities=db.activities
activities_metadata=db.activities_metadata
organizations = db.organizations

db.collection_names()

['organizations',
 'activities',
 'activities_metadata',
 'transactions',
 'organizations_metadata']

In [6]:
print(activities_metadata.count(), activities.count())

4542 764159


# Cache in Memory

In [7]:
all_metadata = {}
metadata_count = activities_metadata.count()

print(datetime.now(), 'Started processing')

for num, metadata_item in enumerate(activities_metadata.find()):
    all_metadata[metadata_item['@w210-key']] = metadata_item

print(datetime.now(), 'Finished processing')

2017-12-31 19:09:08.843641 Started processing
2017-12-31 19:09:08.974730 Finished processing


In [8]:
len(all_metadata)

4542

# Store URLs from Metadata

In [9]:
iati_urls = {}

In [10]:
def add_from_activity_metadata(filename, metadata):
    if 'organization' not in metadata:
        return

    organization = metadata['organization']

    if organization is None:
        return

    urlname = organization['name']
    title = organization['title']

    iati_urls[filename] = (urlname, title)

In [11]:
iati_urls.clear()

for filename, metadata in all_metadata.items():
    add_from_activity_metadata(filename, metadata)

In [12]:
len(iati_urls)

4541

So we see that about 1 organization doesn't have a clean URL we can easily derive. We'll attempt to add it from the activity.

In [13]:
def add_from_activity(filename, activity):
    if 'participating-org' not in activity:
        return

    organization = activity['participating-org']

    if organization is None:
        return

    if type(organization) == list:
        for organization_item in organization:
            if '@role' in organization_item and organization_item['@role'] == 'Implementing':
                organization = organization_item
                break
   
    if '@ref' not in organization:
        return

    organization_ref = organization['@ref']
    title = organization['#text']

    if title is not None:
        iati_urls[filename] = (organization_ref, title)

Let's take a look at what's missing and see how we fill it in.

In [14]:
for filename in (set(all_metadata.keys()) - set(iati_urls.keys())):
    for activity in activities.find({'@w210-key': filename}):
        add_from_activity(filename, activity)

        if filename in iati_urls:
            print(filename, iati_urls[filename])
        else:
            print(filename, None)

dapp-mw-activities ('dapp mw', 'Development Aid from People to People (DAPP) Malawi')


In [15]:
len(iati_urls)

4542

It looks like we were able to drive some additional metadata for that missing organization. Let's go ahead and pickle this file.

In [16]:
with open('iati_urls.pickle', 'wb') as f:
    pickle.dump(iati_urls, f)

## Cache Activities

Bring all the activities out of MongoDB and into memory so that we can process them much faster. Iterating over the MongoDB collection takes minutes. Iterating over an in-memory version containing only the fields we're interested in takes seconds.

However, we need a lot of memory to fit everything, so we'll only include a few of the fields which might be useful when constructing the graph.

* http://iatistandard.org/202/activity-standard/elements/

In [17]:
iati_urls_by_ref = {}

In [18]:
def add_from_activity(item):
    if 'reporting-org' not in activity:
        return

    filename = activity['@w210-key']

    if filename in iati_urls:
        iati_name = iati_urls[filename][0]
    else:
        iati_name = filename[0:filename.find('-')]

    iati_url = 'https://www.iatiregistry.org/publisher/%s' % iati_name

    organization = activity['reporting-org']

    if '@ref' not in organization:
        return

    organization_ref = organization['@ref']

    if organization_ref not in iati_urls_by_ref:
        iati_urls_by_ref[organization_ref] = set([iati_url])
    else:
        iati_urls_by_ref[organization_ref].add(iati_url)

In [19]:
iati_urls_by_ref = {}

print(datetime.now(), 'Started processing')

for activity in activities.find():
    add_from_activity(activity)

print(datetime.now(), 'Finished processing')

2017-12-31 19:09:16.554226 Started processing
2017-12-31 19:10:34.123317 Finished processing


Let's pickle the file.

In [20]:
with open('iati_urls_by_ref.pickle', 'wb') as f:
    pickle.dump(iati_urls_by_ref, f)

# Check Fields (Summary)

Let's get a sense of how populated each field is in the data.

In [21]:
fields = set([
    '@w210-key',
    'iati-identifier',
    'other-identifier',
    'reporting-org',
    'participating-org',
    'activity-status',
    'recipient-country',
    'recipient-region',
    'location',
    'sector',
    'transaction'
])

In [22]:
# Count how many times each of the keys of interest appears
# in our activity data set.

field_counts = { key: 0 for key in fields }
field_keys = field_counts.keys()

print(datetime.now(), 'Started processing')

for activity in activities.find():
    for key in field_keys:
        if key in activity:
            field_counts[key] += 1

print(datetime.now(), 'Finished processing')

field_counts

2017-12-31 19:10:34.164410 Started processing
2017-12-31 19:11:50.248107 Finished processing


{'@w210-key': 764159,
 'activity-status': 746821,
 'iati-identifier': 764094,
 'location': 317755,
 'other-identifier': 186305,
 'participating-org': 762532,
 'recipient-country': 591713,
 'recipient-region': 327262,
 'reporting-org': 764159,
 'sector': 500755,
 'transaction': 729967}

Based on these counts, we'll prefer anything where we can:

* Include the file name
* Include the participating organization
* Include the reporting organization
* Include the transaction information

# Check Fields (One Record)

In [23]:
activity = activities.find_one({ key: {'$ne': None } for key in fields })

In [24]:
activity['iati-identifier']

'XM-DAC-701-8-2012003030'

In [25]:
# http://iatistandard.org/202/activity-standard/iati-activities/iati-activity/sector/
# Vocabulary: http://iatistandard.org/202/codelists/SectorVocabulary/

activity['sector']

{'#text': 'Agricultural water resources',
 '@code': '31140',
 '@percentage': '100',
 '@vocabulary': 'DAC'}

In [26]:
# Ref: http://iatistandard.org/202/activity-standard/iati-activities/iati-activity/participating-org/
# Type: http://iatistandard.org/202/codelists/OrganisationType/

activity['reporting-org']

{'#text': 'Japan International Cooperation Agency',
 '@ref': 'XM-DAC-701-8',
 '@type': '10'}

In [27]:
# Ref: http://iatistandard.org/202/activity-standard/iati-activities/iati-activity/participating-org/
# Role: http://iatistandard.org/202/codelists/OrganisationRole/
# Type: http://iatistandard.org/202/codelists/OrganisationType/

activity['participating-org']

[{'#text': 'PROGRAMA DE DESARROLLO PRODUCTIVO AGRARIO RURAL (AGRORURAL)',
  '@role': 'Accountable',
  '@type': '10'},
 {'#text': 'Japan', '@ref': 'JP', '@role': 'Funding', '@type': '10'},
 {'#text': 'Japan International Cooperation Agency',
  '@ref': 'XM-DAC-701-8',
  '@role': 'Extending',
  '@type': '10'},
 {'#text': 'PROGRAMA DE DESARROLLO PRODUCTIVO AGRARIO RURAL (AGRORURAL)',
  '@role': 'Implementing',
  '@type': '10'}]

In [28]:
# Code: http://iatistandard.org/202/codelists/ActivityStatus/

activity['activity-status']

{'#text': 'Implementation', '@code': '2'}

In [29]:
# Code: http://iatistandard.org/202/codelists/Country/

activity['recipient-country']

{'#text': 'PERU', '@code': 'PE'}

In [30]:
# FinanceType: http://iatistandard.org/202/codelists/FinanceType/
# TransactionType: http://iatistandard.org/202/codelists/TransactionType/

activity['transaction']

{'@generated-datetime': '2015-09-25T09:29:59',
 '@w210-activity': 'XM-DAC-701-8-2012003030',
 '@w210-key': 'jica-489_2014',
 'transaction-date': {'#text': '2014-12-31', '@iso-date': '2014-12-31'},
 'transaction-type': {'#text': 'Disbursement', '@code': 'D'},
 'value': {'#text': '113560000',
  '@currency': 'JPY',
  '@value-date': '2014-12-31'}}

# Check Data

In [31]:
def flatten(item_list):
    if type(item_list) != list:
        return [item_list]

    return_value = []

    # Recursively flatten lists of lists

    for item in item_list:
        if type(item) == list:
            return_value += flatten(item)
        else:
            return_value.append(item)

    return return_value

In [32]:
def get_text(element, attribute):

    if element is None:
        return None

    if attribute not in element:
        return None

    try:
        value = element[attribute]
    except Exception as e:
        print(element, attribute)
        raise e

    if type(value) == dict:
        if '#text' in value:
            return value['#text']

        return None

    if type(value) != list:
        return value

    return [
        item['#text'] if type(item) == dict and '#text' in item else
            None if type(item) == dict else item
        for item in value
    ]

In [33]:
def get_node_list(parent, field_keys):
    value = parent

    for key in field_keys:

        # If we have a dictionary, we simply access the attribute

        if type(value) == dict:
            if key not in value:
                return []

            value = value[key]
            continue

        # If we have something that is neither a dict nor a list, we
        # cannot navigate further down the JSON object, so we were
        # unable to find what we needed.

        if type(value) != list:
            return []

        # If we have a list, then we'll check the key in each element
        # of the list.

        value = [ item[key] for item in value if item is not None and key in item ]

    if value is None:
        return []

    if type(value) != list:
        value = [value]

    return value

In [34]:
def get_value(activity, field_keys):
    node_list = get_node_list(activity, field_keys)

    return_value = []

    for node in node_list:
        ref = get_text(node, '@ref')
        narrative = get_text(node, 'narrative')

        if narrative is not None:
            return_value.append({
                'ref': ref, 'narrative': narrative
            })

        narrative = get_text(node, '@narrative')

        if narrative is not None:
            return_value.append({
                'ref': ref, 'narrative': narrative
            })

    return return_value

In [35]:
def get_values(field_path):
    field_keys = field_path.split('.')

    return_values = []

    for activity in activities.find():
        new_values = get_value(activity, field_keys)

        for value in new_values:
            value['file_id'] = activity['@w210-key']

        return_values += new_values

    return return_values

In [36]:
def get_new_version_values(element_name):
    return pandas.DataFrame(get_values(element_name))

## Reporting Organizations

In [37]:
reporters = get_new_version_values('reporting-org')

In [38]:
reporters.head()

,file_id,narrative,ref
0,unitedstates-bh,Department of Justice,US-GOV-15
1,unitedstates-bh,Department of Justice,US-GOV-15
2,unitedstates-bh,Department of Justice,US-GOV-15
3,unitedstates-bh,Department of Justice,US-GOV-15
4,unitedstates-bh,Department of Homeland Security,US-GOV-19


In [39]:
reporters.tail()

,file_id,narrative,ref
598680,usaid-ua,U.S. Agency for International Development,US-GOV-1
598681,usaid-ua,U.S. Agency for International Development,US-GOV-1
598682,usaid-ua,U.S. Agency for International Development,US-GOV-1
598683,usaid-ua,U.S. Agency for International Development,US-GOV-1
598684,usaid-ua,U.S. Agency for International Development,US-GOV-1


## Other Identifier

In [40]:
others = get_new_version_values('other-identifier.owner-org')

In [41]:
others.head()

,file_id,narrative,ref
0,maec-za,"Ministry of Foreign Affairs and Cooperation, S...",ES-DIR3-E04585801
1,maec-za,"Ministry of Foreign Affairs and Cooperation, S...",ES-DIR3-E04585801
2,maec-za,"Ministry of Foreign Affairs and Cooperation, S...",ES-DIR3-E04585801
3,maec-za,"Ministry of Foreign Affairs and Cooperation, S...",ES-DIR3-E04585801
4,maec-za,"Ministry of Foreign Affairs and Cooperation, S...",ES-DIR3-E04585801


In [42]:
others.tail()

,file_id,narrative,ref
127028,ocha_fts-sudan_2013,UN OCHA Financial Tracking Service,XM-OCHA-FTS
127029,ocha_fts-sudan_2013,UN OCHA Financial Tracking Service,XM-OCHA-FTS
127030,ocha_fts-sudan_2013,UN OCHA Financial Tracking Service,XM-OCHA-FTS
127031,ocha_fts-sudan_2013,UN OCHA Financial Tracking Service,XM-OCHA-FTS
127032,ocha_fts-sudan_2013,UN OCHA Financial Tracking Service,XM-OCHA-FTS


## Participating Organizations

In [43]:
participants = get_new_version_values('participating-org')

In [44]:
participants.head()

,file_id,narrative,ref
0,unitedstates-bh,USA,US-USAGOV
1,unitedstates-bh,Department of Justice,US-GOV-15
2,unitedstates-bh,U.S. Department of State,US-GOV-11
3,unitedstates-bh,Department of Justice - Criminal Division - In...,None
4,unitedstates-bh,USA,US-USAGOV


In [45]:
participants.tail()

,file_id,narrative,ref
2220206,usaid-ua,Enterprise - Non United States Unknown,None
2220207,usaid-ua,U.S. Agency for International Development,US-GOV-11
2220208,usaid-ua,U.S. Agency for International Development,US-GOV-1
2220209,usaid-ua,U.S. Agency for International Development,US-GOV-1
2220210,usaid-ua,Enterprise - Non United States Unknown,None


## Transaction Providers

In [46]:
transaction_providers = get_new_version_values('transaction.provider-org')

In [47]:
transaction_providers.head()

,file_id,narrative,ref
0,free_press_unlimited-798,Free Press Unlimited,NL-KVK-52957535
1,free_press_unlimited-798,Free Press Unlimited,NL-KVK-52957535
2,who-eth,Assessed Contributions,
3,who-eth,Core Voluntary Contributions,
4,who-eth,United States of America,


In [48]:
transaction_providers.tail()

,file_id,narrative,ref
811185,unitedstates-ne2,Millennium Challenge Corporation,US-18
811186,unitedstates-ne2,Millennium Challenge Corporation,US-18
811187,unitedstates-ne2,Millennium Challenge Corporation,US-18
811188,unitedstates-ne2,Millennium Challenge Corporation,US-18
811189,unitedstates-ne2,Millennium Challenge Corporation,US-18


## Transaction Receivers

In [49]:
transaction_receivers = get_new_version_values('transaction.receiver-org')

In [50]:
transaction_receivers.head()

,file_id,narrative,ref
0,free_press_unlimited-798,Free Press Unlimited,NL-KVK-52957535
1,free_press_unlimited-798,Free Press Unlimited,NL-KVK-52957535
2,ec-devco-qre,Associazione Culturale Cooperazione Internazio...,None
3,ec-devco-qre,Associazione Culturale Cooperazione Internazio...,None
4,ec-devco-qre,Commune De Bordeaux,None


In [51]:
transaction_receivers.tail()

,file_id,narrative,ref
1015891,unitedstates-ne2,Niger,None
1015892,unitedstates-ne2,Niger,None
1015893,unitedstates-ne2,Niger,None
1015894,unitedstates-ne2,Niger,None
1015895,unitedstates-ne2,Niger,None


# Check Ref-Names

In [52]:
name_lookup = defaultdict(set)

In [53]:
def get_names_from_organization(item, ref):
    short_names = []

    if '#text' in item:
        short_names.extend([item['#text']])
   
    if 'name' in item:
        get_names_from_organization(item['name'], ref)
   
    if 'narrative' in item:
        narrative = item['narrative']

        if type(narrative) == dict:
            get_names_from_organization(narrative, ref)
        elif type(narrative) == list:
            for subnarrative in narrative:
                get_names_from_organization(subnarrative, ref)
        else:
            short_names.extend([narrative])

    if len(short_names) == 0:
        return

    if ref not in name_lookup:
        name_lookup[ref] = set()

    name_lookup[ref] |= set(short_names)

In [54]:
for organization in organizations.find():
    if 'organisation-identifier' in organization:
        ref = organization['organisation-identifier']

        if type(ref) == dict:
            ref = ref['#text']

        get_names_from_organization(organization, ref)

    if 'reporting-org' in organization:
        reporting_org = organization['reporting-org']

        if '@ref' not in reporting_org:
            continue

        ref = reporting_org['@ref']

        if type(reporting_org) == list:
            for item in reporting_org:
                get_names_from_organization(item, ref)
        elif type(reporting_org) == dict:
            get_names_from_organization(reporting_org, ref)

In [55]:
len(name_lookup)

287

In [56]:
short_names_by_ref = {}
short_names_by_ref.update(name_lookup)

In [57]:
len(short_names_by_ref)

287

## Check Ref to Name Mapping

In [58]:
def get_names_from_activity(df):
    for file_id, ref, narrative in zip(df['file_id'], df['ref'], df['narrative']):
        if ref is None or narrative is None:
            continue

        ref = ref.strip()

        if len(ref) == 0:
            continue

        if type(narrative) != list:
            narrative = narrative.strip()

            if len(narrative) > 0:
                name_lookup[ref].add(narrative)

            continue

        for item in narrative:
            if item is None:
                continue

            item = item.strip()

            if len(item) > 0:
                name_lookup[ref].add(item)

In [59]:
get_names_from_activity(reporters)
get_names_from_activity(others)
get_names_from_activity(participants)
get_names_from_activity(transaction_providers)
get_names_from_activity(transaction_receivers)

In [60]:
len(name_lookup)

10123

In [61]:
multiple_names = []

for ref, names in name_lookup.items():
    if len(names) > 1:
        multiple_names.append({'ref': ref, 'narrative': names})

In [62]:
len(multiple_names)

2302

In [63]:
pandas.DataFrame(multiple_names).head()

,narrative,ref
0,"{Stichting Both Ends, STICHTING BOTH ENDS, Sti...",NL-KVK-41210098
1,"{Theatre for a Change Malawi, TfaC MW, Q4 Expe...",MW-NBM-00139
2,"{UNICEF Kinshasa (Inactive), UNICEF UK, UNITED...",41122
3,"{Y3 Q4 disbursement, Income from DFID Y4 Q1, D...",GB-CHC-327519
4,"{Department Of Interior, Department of Interior}",US-GOV-9


## Check Name to Ref Mapping

In [64]:
ref_lookup = defaultdict(set)

In [65]:
def get_refs(df):
    for file_id, ref, narrative in zip(df['file_id'], df['ref'], df['narrative']):
        if ref is None or narrative is None:
            continue

        ref = ref.strip()

        if len(ref) == 0:
            continue

        if type(narrative) != list:
            narrative = narrative.strip()

            if len(narrative) > 0:
                ref_lookup[narrative].add(ref)

            continue

        for item in narrative:
            if item is None:
                continue

            item = item.strip()

            if len(item) > 0:
                ref_lookup[item].add(ref)

In [66]:
get_refs(reporters)
get_refs(others)
get_refs(participants)
get_refs(transaction_providers)
get_refs(transaction_receivers)

In [67]:
len(ref_lookup)

28588

In [68]:
multiple_refs = []

for name, refs in ref_lookup.items():
    if len(refs) > 1:
        multiple_refs.append({'narrative': name, 'ref': refs})

In [69]:
len(multiple_refs)

2463

In [70]:
pandas.DataFrame(multiple_refs).head()

,narrative,ref
0,Free Press Unlimited,"{504222, NL-KVK-52957535}"
1,Transparency International Secretariat,"{XM-DAC-21033, 21033}"
2,United Nations Population Fund,"{XM-DAC-41119, 42003, XM-OCHA-FTS1171, 42001, ..."
3,World Health Organization,"{XM-OCHA-FTS122, 02358, 42001, 10000, XM-DAC-928}"
4,CAFOD,"{22000, GB-CHC-285776}"


## Store as Pickled Files

In [71]:
with open('short_names_by_ref.pickle', 'wb') as f:
    pickle.dump(short_names_by_ref, f)

In [72]:
with open('lookup_by_ref.pickle', 'wb') as f:
    pickle.dump(name_lookup, f)

In [73]:
with open('lookup_by_name.pickle', 'wb') as f:
    pickle.dump(ref_lookup, f)

# Backup Files

In [74]:
!rm -f lookup.tar.gz
!tar -cf lookup.tar iati_urls_by_ref.pickle short_names_by_ref.pickle lookup_by_ref.pickle lookup_by_name.pickle
!gzip lookup.tar

In [75]:
s3_bucket = 'mdang.w210'

In [76]:
!aws s3 cp lookup.tar.gz s3://{s3_bucket}/ --acl public-read

upload: ./lookup.tar.gz to s3://mdang.w210/lookup.tar.gz
